# Day 16 - Maze

# Read file as lines

In [1]:
TEST = False

INPUT_FN = "input_test.txt"
TEST_SOLUTION = 7036

#INPUT_FN = "input_test2.txt"
#TEST_SOLUTION = 11048


if not TEST:
    INPUT_FN = "input.txt"

with open(INPUT_FN) as f:
    lines = [l.rstrip() for l in f]

if lines[-1]=='':
    lines=lines[:,-1]

In [2]:
lines

['#############################################################################################################################################',
 '#.........#.............#.....#.....#.....#.........#...#.............#...#.............#.........#.....#...................#.....#.#......E#',
 '#.#######.#.###########.#.###.###.#.#.#.#.#.#.#.###.###.#.###########.#.###.#####.#######.#.###.#.###.#.#.###########.#.###.###.#.#.#.###.###',
 '#.......................#.#.....#.#.#.#.#.#...#...#...#.#.#.......#...#.......................#.#...............#...........................#',
 '#.#.#.#.#####.#####.#.###.#####.#.#.###.#.###.###.###.#.#.#.#.#.###.###.#.###.#.#.#.#########.#.#####.#######.#.#.#.###.#.###.#.#.###.#####.#',
 '#...#.#.#...#.#...#...#...#...#...#...#.#.........#.....#.#.#.#.#...#...#.#...#.#.....#.....#.#.....#.....#.#.#.#.#.....#.............#.....#',
 '#.###.###.#.###.#.#.#####.#.#.#######.#.#####.#.#######.#.#.#.###.###.###.#.###########.###.#######.#.###.#.#.#.#.#####.#.

In [3]:
MAP=lines

Find start and end

In [4]:
start_pos=None
end_pos = None

for r0,l0 in enumerate(lines):
    if 'S' in l0:
        start_pos = (r0,l0.index('S'))
    if 'E' in l0:
        end_pos = (r0,l0.index('E'))


In [5]:
print(f"{start_pos=}, {end_pos=}")

start_pos=(139, 1), end_pos=(1, 139)


In [6]:
def printd(s):
    if TEST:
        print(s)

Suggested algorithm to solve

BFS whereby visited locations must include the direction of travel

Store the lowest score for each location visited. Have a dictionary to store these.

In [7]:
from collections import deque


best_scores = dict()
visited = []
robot_pos = (*start_pos, 0,+1)
best_scores[robot_pos]= 0
bfs_q = deque()
bfs_q.append(robot_pos)

iter=0

while True:
    printd(f"{iter=} , {bfs_q=}")
    if len(bfs_q)==0:
        # no more nodes to visit
        printd("No more nodes to visit")
        break

    curr_pos = bfs_q.popleft()
    curr_score = best_scores[curr_pos]

    printd(f"{curr_pos=} , {curr_score=}")

    #Check is end position
    if MAP[curr_pos[0]][curr_pos[1]] == 'E':
        printd("End node")
        continue # skip the rest

    #Options are, to move forward or to turn clockwise or anticlockwise
    new_poss = []

    #move forward need to check if there is a wall at new position
    try_pos = (curr_pos[0]+curr_pos[2], curr_pos[1]+curr_pos[3])
    printd(f"move forward to {try_pos=} ?")

    if MAP[try_pos[0]][try_pos[1]] != '#':
        # ok
        printd("can move forward")
        new_poss.append( ( (*try_pos, curr_pos[2], curr_pos[3]) , curr_score+1) )
    
    # rotations
    new_poss.append( ((curr_pos[0], curr_pos[1], -curr_pos[3], curr_pos[2]), curr_score+1000) ) # clockwise
    new_poss.append( ((curr_pos[0], curr_pos[1], curr_pos[3], -curr_pos[2]) , curr_score+1000) ) # anti-clockwise

    # for all these new positions to consider, check if they have been visited
    for npos0,sc0 in new_poss:
        #position visited?
        if npos0 in visited:
            # compare scores
            bsc0 = best_scores[npos0]
            
            if sc0<bsc0:
                best_scores[npos0] = sc0

                # re-add position to queue to recalculate any scores after this
                bfs_q.append(npos0)
        else:
            #add this as a new visited place
            visited.append(npos0)
            best_scores[npos0] = sc0
            bfs_q.append(npos0)
    

    iter+=1

This algorithm above does not work well. It is slow because it checks for too many paths. Dijkstra algorithm may be better suited.

There are probably many end positions in the score dictionary, with different directions. I will pick the lowest

In [11]:
best_end_scores = []
for k,sc0 in best_scores.items():
    if k[0] == end_pos[0] and k[1]==end_pos[1]:
        print(f"end score {k=} , {sc0=}")
        best_end_scores.append(sc0)


end score k=(1, 139, 0, 1) , sc0=88468


In [12]:
result = min(best_end_scores)
if TEST:
    assert result == TEST_SOLUTION

In [13]:
result

88468

94428 is not the right answer. Answer is too high.

88468 is the correct answer